# Imports

In [ ]:
import os
import pandas as pd
from pathlib import Path
from pprint import pprint
from scipy.io import loadmat
from typing import Dict, Tuple
from typing import Any

from config import DATA_PATH


def load_matlab_file_as_dict(filename: str) -> Dict[str, Any]:
    loaded_dict: Dict[str, Any] = loadmat(os.path.join(DATA_PATH, filename))
    # pprint(loaded_dict.keys())
    return loaded_dict


from dataclasses import dataclass


@dataclass
class Campo:
    name: str
    cultivated: int
    grid: Tuple[int, int]
    lat: float
    lon: float
    init_water: float


import re


def dms_to_dd(dms_str):
    """
    Convierte un string en formato DMS (ej: '30°53\'23,37"') a decimal.
    """
    # Reemplazar coma por punto (por decimales en español)
    dms_str = dms_str.replace(",", ".")

    # Extraer números con regex
    match = re.match(r"(\d+)°(\d+)'([\d\.]+)\"", dms_str)
    if not match:
        raise ValueError(f"Formato no reconocido: {dms_str}")

    degrees, minutes, seconds = match.groups()
    degrees = float(degrees)
    minutes = float(minutes)
    seconds = float(seconds)

    return degrees + minutes / 60 + seconds / 3600

# Carga de archivos para simulación

### Matrices de latitudes, longitudes y máscaras de un campo $C_{ij}$

In [ ]:
cultivo = load_matlab_file_as_dict("mat_maiz_2021_lowres.mat")[
    "clase_maiz_2021_lowres"
]  # máscara


cultivos_lat = load_matlab_file_as_dict("mat_maiz_2021_lat_lowres.mat")["lat_lowres"]


cultivos_lon = load_matlab_file_as_dict("mat_maiz_2021_lon_lowres.mat")["lon_lowres"]


N_lat = cultivo.shape[0]
N_lon = cultivo.shape[1]

print(f"Latitudes lowres: {cultivos_lat.shape}")
print(cultivos_lat[0:2, 0:2])
print(f"\nLongitudes lowres: {cultivos_lon.shape}")
print(cultivos_lon[0:2, 0:2])
print(f"\nMascara: {cultivo.shape}")
print(cultivo[0:3, 0:3])

### Matriz de porcentaje de agua inicial en los campos

In [ ]:
porc_agua_inicial = (
    load_matlab_file_as_dict("mat_aguadisp_saocom_maiz_2021-2022_2.mat")["a_disp_campo"]
    / 100
)
print(f"Porcentaje de agua inical: {porc_agua_inicial.shape}")
print(porc_agua_inicial[:2, :2])

### Matriz de días desde la siembra (DDS)

In [ ]:
dds_est = load_matlab_file_as_dict("mat_dds_maiz_est_lowres.mat")["dds_est"]
print(f"Dias desde la siembra estimacion: {dds_est.shape}")
print(dds_est[:2, :2])

### Matrices de agua observada y fechas de observación

In [ ]:
agua_obs = load_matlab_file_as_dict("agua_obs_m.mat")["agua_obs"]
print(f"agua observada: {agua_obs.shape}")
print(agua_obs[:2, :2])

In [ ]:
# Harcodeadeadas por gonazalo.
fechas_aguas_m = [
    "2021-11-19",
    "2021-11-29",
    "2021-12-09",
    "2021-12-19",
    "2021-12-29",
    "2022-01-08",
    "2022-01-18",
    "2022-01-28",
    "2022-02-07",
    "2022-02-17",
    "2022-03-10",
    "2022-03-20",
    "2022-03-30",
    "2022-04-10",
    "2022-04-20",
    "2022-05-01",
    "2022-05-14",
]

print(fechas_aguas_m.__len__())

### Ejemplo de tarjeta de campo

In [ ]:
i, j = 2, 1
campo_ij = Campo(
    name="Ejemplo",
    lat=cultivos_lat[i, j],
    lon=cultivos_lat[i, j],
    grid=(i, j),
    cultivated=cultivo[i, j],
    init_water=porc_agua_inicial[i, j],
)
print(campo_ij)

### Cargar coordenadas de maiz 2021

In [ ]:
# --- Config ---
XLSX_PATH = Path(DATA_PATH) / "fechasiembra_2020.xlsx"  # ajusta si hace falta
SHEET_NAME = "Maiz"
df_coordenadas_maiz = pd.read_excel(XLSX_PATH, sheet_name=SHEET_NAME)
df_coordenadas_maiz

In [ ]:
df_coordenadas_maiz["lat_dd"] = -1 * df_coordenadas_maiz["lat"].apply(dms_to_dd)
df_coordenadas_maiz["long_dd"] = -1 * df_coordenadas_maiz["long"].apply(dms_to_dd)

In [ ]:
mat_coord_m = df_coordenadas_maiz[["lat_dd", "long_dd"]]
lats_lons = mat_coord_m

In [ ]:
mat_fs_m = df_coordenadas_maiz["FS"]
mat_fs = mat_fs_m